In [1]:
!pip install langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0 PyPDF2==3.0.1 python-dotenv==1.0.0 streamlit==1.18.1 openai==0.27.6 faiss-cpu==1.7.4 altair==4 tiktoken==0.4.0

  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------ ------- 786.4/981.5 kB 3.7 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached tzlocal-5.2-py3-none-any.whl.metadata (7.8 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.110.0 requires starlette<0.37.0,>=0.36.3, which is not installed.
tensorflow 2.6.0 requires clang~=5.0, which is not installed.
tensorboard 2.6.0 requires google-auth<2,>=1.6.3, but you have google-auth 2.29.0 which is incompatible.
tensorflow 2.6.0 requires absl-py~=0.10, but you have absl-py 2.1.0 which is incompatible.
tensorflow 2.6.0 requires flatbuffers~=1.12, but you have flatbuffers 20210226132247 which is incompatible.


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.4 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.4 MB 4.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/9.4 MB 4.1 MB/s eta 0:00:02
   ------------- -------------------------- 3.1/9.4 MB 4.0 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.4 MB 4.0 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.4 MB 4.0 MB/s eta 0:00:02
   ----------------------- ---------------- 5.5/9.4 MB 3.9 MB/s eta 0:00:01
   -------------------------- ------------- 6.3/9.4 MB 3.9 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.4 MB 3.7 MB/s eta 0:00:01
   ------------------------------ --------- 7.1/9.4 MB 3.7 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.4 MB 3.6 MB/s eta 0:00:01
   ------------------------------------- -- 8.9/9.4 MB 3.7 MB/s eta 0:00:01
   ----------------

In [8]:
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [16]:
# %%writefile app.py
#Create VectorDB Function
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA


embeddings = HuggingFaceEmbeddings()


def Create_vectordb(PDF,embeddings):
  # loading the document
  text = ""
  pdf_reader = PdfReader(PDF)
  for page in pdf_reader.pages:
    text += page.extract_text()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)

  texts = text_splitter.split_text(text)
  vectordb = FAISS.from_texts(texts=texts
                              ,embedding=embeddings)
  return vectordb


2024-12-07 20:30:20.422 INFO    sentence_transformers.SentenceTransformer: Use pytorch device_name: cpu
2024-12-07 20:30:20.423 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [17]:
# %%writefile app.py -a

def Load_vectordb(Folder_Path,embeddings):
  return FAISS.load_local(folder_path=Folder_Path,embeddings=embeddings,allow_dangerous_deserialization=True)




In [18]:
# %%writefile app.py -a

def save_vectordb(vectordb,Folder_Path):
   vectordb.save_local(folder_path=Folder_Path)
   print("save successful")



In [19]:
# %%writefile app.py -a

def Update_VectorDB(Folder_Path,embeddings,Data,Type):
  vectordb = Load_vectordb(Folder_Path,embeddings)

  # splitting into text chunks
  if Type== 1:
    text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=300,
    length_function=len)
    text = ""
    pdf_reader = PdfReader(Data)
    for page in pdf_reader.pages:
      text += page.extract_text()
    texts = text_splitter.split_text(text)
    vectordb.add_texts(texts)
    print("VectorDB updated")
  if Type == 2:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
    loader = WebBaseLoader(web_paths=(Data,),
                          bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                              class_=("mw-body"))))
    text_documents = loader.load()

    # Convert the list to an iterable
    documents = text_splitter.split_documents(text_documents)
    vectordb.add_documents(documents)
  return vectordb




In [20]:
# %%writefile app.py -a

def load_LLM_groq(vectordb,Model_name):
  retriever = vectordb.as_retriever()
  llm = ChatGroq(
    model=Model_name,
    temperature=0)
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)
  return qa_chain



In [21]:
def load_LLM_groq(vectordb,Model_name):
  retriever = vectordb.as_retriever()
  llm = ChatGroq(
    model=Model_name,
    temperature=0)
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)
  return qa_chain

In [22]:
embeddings = HuggingFaceEmbeddings()




vectordb = Create_vectordb("Ancient History of Egypt.pdf",embeddings)
print("created vectorDB")
save_vectordb(vectordb,"VectorDB")
vectordb = Load_vectordb("VectorDB",embeddings)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/History_of_Egypt",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/State_of_Palestine",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/History_of_Palestine",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_history",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_culture",2)
vectordb = Update_VectorDB("VectorDB",embeddings,"https://en.wikipedia.org/wiki/Arabic_religion",2)
save_vectordb(vectordb,"VectorDB")

2024-12-07 20:30:24.351 INFO    sentence_transformers.SentenceTransformer: Use pytorch device_name: cpu
2024-12-07 20:30:24.352 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-12-07 20:30:27.929 INFO    faiss.loader: Loading faiss with AVX2 support.
2024-12-07 20:30:27.930 INFO    faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2024-12-07 20:30:27.930 INFO    faiss.loader: Loading faiss.
2024-12-07 20:30:27.998 INFO    faiss.loader: Successfully loaded faiss.


created vectorDB
save successful
save successful


In [23]:
# print(vectordb.similarity_search("WHAT IS AI"))
print(vectordb.similarity_search("WHAT IS MENTAL HEALTH?"))
print(vectordb.similarity_search("WHAT IS history of egypt"))

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Arabic_religion'}, page_content='Other reasons this message may be displayed:\n\nIf a page was recently created here, it may not be visible yet because of a delay in updating the database; wait a few minutes or try the purge function.\nTitles on Wikipedia are case sensitive except for the first character; please check alternative capitalizations and consider adding a redirect here to the correct title.\nIf the page has been deleted, check the deletion log, and see Why was the page I created deleted?\n\n\n\nRetrieved from "https://en.wikipedia.org/wiki/Arabic_religion"'), Document(metadata={'source': 'https://en.wikipedia.org/wiki/Arabic_religion'}, page_content='Arabic religion\n\n\n\nAdd languages\n\n\n\n\t\t\t\t\t\tPage contents not supported in other languages.\n\t\t\t\t\t\n\n\n\n\n\n\n\n\n\n\nArticleTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n

In [24]:
import langchain
langchain.__version__

'0.2.11'

In [25]:
os.environ["GROQ_API_KEY"] = "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"

In [26]:
qa_chain=load_LLM_groq(vectordb,"llama-3.1-70b-versatile")
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''

In [28]:
# # invoke the qa chain and get a response for user query
query = prompt + """
what is the history of palestine?
"""
response = qa_chain.invoke({"query": query})
print(response["result"])

2024-12-07 20:44:21.812 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Palestine's history spans over 5,000 years, with various empires and civilizations ruling the region. Here's a brief overview:

* Ancient Period (3000 BC - 636 AD): Canaanites, Philistines, Israelites, and other tribes inhabited the region.
* Roman and Byzantine Period (636 - 638 AD): Palestine was a Roman province, then a Byzantine province.
* Islamic Period (638 - 1099 AD): Arab conquest, followed by the Umayyad and Abbasid Caliphates.
* Crusader Period (1099 - 1187 AD): Christian rule, then recaptured by Saladin.
* Ayyubid and Mamluk Period (1187 - 1517 AD): Muslim rule, with the Ayyubid and Mamluk dynasties.
* Ottoman Period (1517 - 1917 AD): Palestine was an Ottoman province.
* British Mandate (1917 - 1948 AD): Britain ruled Palestine, with the 1947 UN Partition Plan leading to the establishment of Israel.
* Modern Period (1948 - present): The Israeli-Palestinian conflict continues, with ongoing disputes over territory, identity, and statehood.


In [35]:
query = prompt + """
اخبرني عن اسرئيل
"""
response = qa_chain.invoke({"query": query})
print(response["result"])

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-05 21:16:23.519 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


لا أملك معلومات كافية عن تاريخ إسرائيل، ولكن يمكنني أن أخبرك أن تاريخ إسرائيل الحديث يبدأ بعد عام 1948، عندما أعلنت إسرائيل استقلالها عن الانتداب البريطاني على فلسطين.


In [29]:
%%writefile app.py -a
import streamlit as st

vectordb= Load_vectordb("VectorDB",embeddings)
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''
# Title of the Streamlit app
st.title("LLM RAG Chatbot")

# Selection box to choose between OpenAI and Groq models
model_option = st.selectbox('Choose a model:', ('OpenAI', 'Groq'))

# Input box for user query
user_input = st.text_input("Ask your question:")
user_input = prompt + user_input
# Add OpenAI and Groq API keys to the environment
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
os.environ["GROQ_API_KEY"] = "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"

# Button to submit the query
if st.button('Get Answer'):

    # Check which model the user selected
    if model_option == 'OpenAI':
        # Load OpenAI model (for example: GPT-4)
        qa_chain = load_LLM_openai(vectordb, model_name="gpt-4o")
        response = qa_chain.invoke({"query": user_input})
        st.write(f"OpenAI's response: {response['result']}")

    elif model_option == 'Groq':
        # Load Groq model
        qa_chain=load_LLM_groq(vectordb,"llama-3.2-11b-text-preview")
        response = qa_chain.invoke({"query": user_input})
        st.write(f"Groq's response: {response['result']}")


Appending to app.py


In [30]:
%%writefile app.py -a
import streamlit as st
# Initialize session state to store chat history and API keys
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "openai_api_key" not in st.session_state:
    st.session_state["openai_api_key"] = ""
if "groq_api_key" not in st.session_state:
    st.session_state["groq_api_key"] = ""

# Sidebar for Model Selection and API Key Input
st.sidebar.title("Configuration")

# API Key Input Fields
st.sidebar.subheader("Enter API Keys")
st.session_state["openai_api_key"] = st.sidebar.text_input("OpenAI API Key", type="password")
st.session_state["groq_api_key"] = st.sidebar.text_input("Groq API Key", type="password")

# Model Selection Dropdown
model_option = st.sidebar.selectbox('Choose a model:', ('OpenAI', 'Groq'))

# Set API keys in environment variables if provided

# Load FAISS vectordb (assuming you have already created the FAISS index)
vectordb= Load_vectordb("VectorDB",embeddings)
prompt = '''
you are an expert historian who knows about egypt ,palastine and arabic history answer the questions as short as possible/n
'''

# Chatbot Interface: Title and Chat History
st.title("LLM RAG Chatbot")

# Display chat history
st.write("### Chat History:")
for message in st.session_state["messages"]:
    st.write(message)

# Input box for user query
user_input = st.text_input("You:", key="user_input")

# Button to send the query
if st.button('Send'):
    # Append user input to chat history
    st.session_state["messages"].append(f"You: {user_input}")
    user_input = prompt + user_input

    # Ensure that the appropriate API key is provided
    if model_option == 'OpenAI' and not st.session_state["openai_api_key"]:
        st.error("Please enter your OpenAI API key.")
    elif model_option == 'Groq' and not st.session_state["groq_api_key"]:
        st.error("Please enter your Groq API key.")
    else:
        # Process the query using the selected model
        if model_option == 'OpenAI':
          if st.session_state["openai_api_key"]:
            os.environ["OPENAI_API_KEY"] = st.session_state["openai_api_key"]
            qa_chain = load_LLM_openai(vectordb, model_name="gpt-4")
            response = qa_chain({"query": user_input})
            bot_response = f"OpenAI: {response['answer']}"

        elif model_option == 'Groq':
          if st.session_state["groq_api_key"]:
            os.environ["GROQ_API_KEY"] = st.session_state["groq_api_key"]
            qa_chain=load_LLM_groq(vectordb,"llama-3.2-90b-text-preview")
            response = qa_chain({"query": user_input})
            bot_response = f"Groq: {response['result']}"

        # Append the bot's response to the chat history
        st.session_state["messages"].append(bot_response)

        # Clear input box and refresh chat
        st.experimental_rerun()  # Refresh to display the new message


Appending to app.py


In [31]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
changed 22 packages in 2s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [32]:
!wget -q -O - ipv4.icanhazip.com

35.185.230.111


In [33]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.230.111:8501

your url is: https://ripe-pandas-wait.loca.lt
  Stopping...
^C
